In [44]:
class Value:
    def __init__(self, data, _children = (), _op = "", label =""):
        self.data = data
        self._prev = _children
        self._op = _op
        self.label = label
        self._backward = lambda: None
        self.grad = 0.0

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), "+")

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), "*")
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out 
    
    def __pow__(self, other):
        assert isinstance(other,(int, float))
        out = Value(self.data**other, (self,), f"**{other}")

        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward = _backward

        return out 
    
    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self, ), "ReLU")

        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward
        
        return out
    
    def __rmul__(self, other):
        return self * other
    
    def __radd__(self, other):
        return self + other 
    
    def __repr__(self):
        return f"Value(data = {self.data})"
    

In [45]:
b = Value(3.0)
c = b**2
d = c+c
d.grad = 1
d._backward()
c._backward() 
b.grad

12.0

In [37]:
a = Value(2.0)
b = Value(3.0)

c = a + b
c.grad = 1

c._backward()

1.0

In [35]:
c._backward()

AttributeError: 'Value' object has no attribute 'other'

In [32]:
a = Value(2.0)
b = Value(3.0)

c = a + b
d = Value(6.0)

e = c**2
e

Value(data = 25.0)

In [11]:
import random 

class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, -1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))

    def __call__(self, x):
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()

        return out
    
    def parameters(self):
        return self.w + [self.b]

    
class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]

        return outs[0] if len(outs) ==1 else outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.paramters()]

class MLP:
    def __init__(self, nin, nouts):
        sz = [nin]+nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(nouts)]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)

    def paramters(self):
        return [p for layer in self.layers for p in layer.parameters()]
